# modify_tensor vs. create_tensor
- modify_tensor 는 Tensor를 입력으로 받아서 어떤 수정이나 변경을 함
- create_tensor 는 Tensor 입력 없이 tensor를 만드는 것들

In [1]:
import tensorflow as tf
tf.__version__

'1.15.2'

In [2]:
import numpy as np

In [3]:
sess = tf.Session()

- reform
- mutating

# reform

## reshape
https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/reshape

In [43]:
tsr = [1, 2, 3, 4, 5, 6, 7, 8, 9]
res = tf.reshape(tsr, [3, 3]) 

sess.run(res)

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]], dtype=int32)

## squeeze

### squeeze 로 1 dim 없애기

In [8]:
tgt = np.ndarray(shape=(1,2,1,1,3), dtype=float)
tsr = tf.convert_to_tensor(tgt)
goal = tf.squeeze(tsr)
res = sess.run(goal)
res.shape

(2, 3)

### squeeze 로 특정 axis dimension 없애기

In [7]:
tgt = np.ndarray(shape=(1,2,1,1,3), dtype=float)
tsr = tf.convert_to_tensor(tgt)
goal = tf.squeeze(tsr, axis=(2,3))
res = sess.run(goal)
res.shape

(1, 2, 3)

## expand_dims
https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/expand_dims

In [56]:
# 't' is a tensor of shape [2]
t = np.ndarray(2)
print(tf.expand_dims(t, 0).shape)  # [1, 2]
print(tf.expand_dims(t, 1).shape)  # [2, 1]
print(tf.expand_dims(t, -1).shape)  # [2, 1]

# 't2' is a tensor of shape [2, 3, 5]
t2 = np.ndarray(shape=(2,3,5))
print(tf.expand_dims(t2, 0).shape)  # [1, 2, 3, 5]
print(tf.expand_dims(t2, 2).shape)  # [2, 3, 1, 5]
print(tf.expand_dims(t2, 3).shape)  # [2, 3, 5, 1]

(1, 2)
(2, 1)
(2, 1)
(1, 2, 3, 5)
(2, 3, 1, 5)
(2, 3, 5, 1)


## concat

In [150]:
t1 = [[1, 2, 3], [4, 5, 6]]
t2 = [[7, 8, 9], [10, 11, 12]]
res1 = tf.concat([t1, t2], 0)  # [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]]
res2 = tf.concat([t1, t2], 1)  # [[1, 2, 3, 7, 8, 9], [4, 5, 6, 10, 11, 12]]

print(sess.run(res1))
print(sess.run(res2))

[[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]]
[[ 1  2  3  7  8  9]
 [ 4  5  6 10 11 12]]


In [151]:
t1 = [[[1, 2], [2, 3]], [[4, 4], [5, 3]]]
t2 = [[[7, 4], [8, 4]], [[2, 10], [15, 11]]]
res = tf.concat([t1, t2], -1)

sess.run(res)

array([[[ 1,  2,  7,  4],
        [ 2,  3,  8,  4]],

       [[ 4,  4,  2, 10],
        [ 5,  3, 15, 11]]], dtype=int32)

## slice

```
tf.slice(
    input_,
    begin,
    size,
    name=None
)
```

In [49]:
t = tf.constant([[[1, 1, 1], [2, 2, 2]],
                 [[3, 3, 3], [4, 4, 4]],
                 [[5, 5, 5], [6, 6, 6]]])

res1 = tf.slice(t, [1, 0, 0], [1, 1, 3])  # [[[3, 3, 3]]]

print(sess.run(res1))

[[[3 3 3]]]


In [47]:
res2 = tf.slice(t, [1, 0, 0], [1, 2, 3])  # [[[3, 3, 3],
                                          #   [4, 4, 4]]]
print(sess.run(res2))

[[[3 3 3]
  [4 4 4]]]


In [48]:
res3 = tf.slice(t, [1, 0, 0], [2, 1, 3])  # [[[3, 3, 3]],
                                          #  [[5, 5, 5]]]
print(sess.run(res3))

[[[3 3 3]]

 [[5 5 5]]]


## scan

### reduce 에 map 같은 역할이 섞임
callable 은 두가지 인자를 받는데,
- 첫번째 a 는 accumulater, 
- 두번째 x 는 elem 입력이다.

In [92]:
elems = np.array([1, 2, 3, 4, 5, 6])
sum = tf.scan(lambda a, x: a + x, elems)

sess.run(sum)

array([ 1,  3,  6, 10, 15, 21])

### `reverse=True` 는 뒤에서 부터 적용한다.

In [95]:
elems = np.array([1, 2, 3, 4, 5, 6])
sum = tf.scan(lambda a, x: a + x, elems, reverse=True)

sess.run(sum)

array([21, 20, 18, 15, 11,  6])

### 당연하게도 initializer 는 처리될 callable 의 출력과 같은 차원이여야 한다
initializer 는 accumulater 의 초기값이니까 말이다

In [107]:
elems = np.array([1, 2, 3, 4, 5, 6])
initializer = np.array(100)
sum = tf.scan(lambda a, x: a + x, elems, initializer)

sess.run(sum)

array([101, 103, 106, 110, 115, 121])

### initializer 가 0 인 경우

In [108]:
elems = np.array([1, 2, 3, 4, 5, 6])
initializer = np.array(0)
sum = tf.scan(lambda a, x: a + x, elems, initializer)

sess.run(sum)

array([ 1,  3,  6, 10, 15, 21])

### elems 는 하나의 tensor 이겠지만 

In [119]:
el0 = np.array([[1, 2, 3, 4, 5, 6]])
el1 = np.array(el0 + 1)

elems = np.concatenate((el0, el1), axis=None)

print(elems)

initializer = np.array(0)
sum = tf.scan(lambda a, x: a + x, elems, initializer)

sess.run(sum)

[1 2 3 4 5 6 2 3 4 5 6 7]


array([ 1,  3,  6, 10, 15, 21, 23, 26, 30, 35, 41, 48])

In [121]:
el0 = np.array([1, 2, 3, 4, 5, 6])
el1 = np.array(el0 + 1)

elems = np.concatenate((el0, el1), axis=0)
print(elems)

initializer = np.array(0)
sum = tf.scan(lambda a, x: a + x, elems, initializer)

sess.run(sum)

[1 2 3 4 5 6 2 3 4 5 6 7]


array([ 1,  3,  6, 10, 15, 21, 23, 26, 30, 35, 41, 48])

### 때로 sequence of tensors 가 되며, 이 multi elems 는 x[0], x[1] 이렇게 참조해서 계산에 이용할 수 있다

In [124]:
elems = np.array([1, 2, 3, 4, 5, 6])
initializer = np.array(0)
sum_one = tf.scan(
    lambda a, x: x[0] - x[1], (elems + 1, elems), initializer)

sess.run(sum_one)

array([1, 1, 1, 1, 1, 1])

In [102]:
elems = np.array([1, 2, 3, 4, 5, 6])
initializer = np.array(0)
sum_one = tf.scan(
    lambda a, x: x[0] - x[1] + a, (elems + 1, elems), initializer)

sess.run(sum_one)

array([1, 2, 3, 4, 5, 6])

In [143]:
el0 = np.array([1, 2, 3, 4, 5, 6])
el1 = np.array(el0 + 1)

elems = [el0, el1]
print('elems:\n', elems)

initializer = np.array(0)

def fn(a,x):
    print('xs: \n', x[0],'\n', x[1])
    return x[1] - x[0]

sum = tf.scan(fn, elems, initializer)

sess.run(sum)

elems:
 [array([1, 2, 3, 4, 5, 6]), array([2, 3, 4, 5, 6, 7])]
xs: 
 Tensor("scan_43/while/TensorArrayReadV3:0", shape=(), dtype=int64) 
 Tensor("scan_43/while/TensorArrayReadV3_1:0", shape=(), dtype=int64)


array([1, 1, 1, 1, 1, 1])

## map_fn

```
tf.map_fn(
    fn,
    elems,
    dtype=None,
    parallel_iterations=None,
    back_prop=True,
    swap_memory=False,
    infer_shape=True,
    name=None
)
```

In [51]:
elems = np.array([1, 2, 3, 4, 5, 6])
squares = tf.map_fn(lambda x: x * x, elems)
# squares == [1, 4, 9, 16, 25, 36]

sess.run(squares)

array([ 1,  4,  9, 16, 25, 36])

## foldl
reduce 같은

In [ ]:
elems = tf.constant([1, 2, 3, 4, 5, 6])
sum = foldl(lambda a, x: a + x, elems)
# sum == 21

## foldr
reduce 같은

In [ ]:
elems = [1, 2, 3, 4, 5, 6]
sum = foldr(lambda a, x: a + x, elems)
# sum == 21

## repeat

### repeats 에 따른 반복

In [59]:
res = tf.repeat(['a', 'b', 'c'], repeats=[3, 0, 2], axis=0)

sess.run(res)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


array([b'a', b'a', b'a', b'c', b'c'], dtype=object)

### axis 에 따른 차이

In [61]:
res = tf.repeat([[1, 2], [3, 4]], repeats=[2, 3], axis=0)

sess.run(res)

array([[1, 2],
       [1, 2],
       [3, 4],
       [3, 4],
       [3, 4]], dtype=int32)

In [62]:
res = tf.repeat([[1, 2], [3, 4]], repeats=[2, 3], axis=1)

sess.run(res)

array([[1, 1, 2, 2, 2],
       [3, 3, 4, 4, 4]], dtype=int32)

### scala input

In [63]:
res = tf.repeat(3, repeats=4)

sess.run(res)

array([3, 3, 3, 3], dtype=int32)

### scala repeats

In [65]:
res = tf.repeat([[1,2], [3,4]], repeats=2)

sess.run(res)

array([1, 1, 2, 2, 3, 3, 4, 4], dtype=int32)

## reverse

In [88]:
tensor = tf.convert_to_tensor([[[[ 0,  1,  2,  3],
                 [ 4,  5,  6,  7],
                 [ 8,  9, 10, 11]],
                [[12, 13, 14, 15],
                 [16, 17, 18, 19],
                 [20, 21, 22, 23]]]])

print('original: \n', sess.run(tensor), '\n\n')
print(tensor.shape)

original: 
 [[[[ 0  1  2  3]
   [ 4  5  6  7]
   [ 8  9 10 11]]

  [[12 13 14 15]
   [16 17 18 19]
   [20 21 22 23]]]] 


(1, 2, 3, 4)


In [81]:
reversed_t = tf.reverse(tensor, axis=[0])

print(sess.run(reversed_t))

[[[[ 0  1  2  3]
   [ 4  5  6  7]
   [ 8  9 10 11]]

  [[12 13 14 15]
   [16 17 18 19]
   [20 21 22 23]]]]


In [76]:
reversed_t = tf.reverse(tensor, axis=[1])

print(sess.run(reversed_t))

[[[[12 13 14 15]
   [16 17 18 19]
   [20 21 22 23]]

  [[ 0  1  2  3]
   [ 4  5  6  7]
   [ 8  9 10 11]]]]


In [77]:
reversed_t = tf.reverse(tensor, axis=[2])

print(sess.run(reversed_t))

[[[[ 8  9 10 11]
   [ 4  5  6  7]
   [ 0  1  2  3]]

  [[20 21 22 23]
   [16 17 18 19]
   [12 13 14 15]]]]


In [78]:
reversed_t = tf.reverse(tensor, axis=[3])

print(sess.run(reversed_t))

[[[[ 3  2  1  0]
   [ 7  6  5  4]
   [11 10  9  8]]

  [[15 14 13 12]
   [19 18 17 16]
   [23 22 21 20]]]]


### multi axis

In [84]:
reversed_t = tf.reverse(tensor, axis=[1,3])

print(sess.run(reversed_t))

[[[[15 14 13 12]
   [19 18 17 16]
   [23 22 21 20]]

  [[ 3  2  1  0]
   [ 7  6  5  4]
   [11 10  9  8]]]]


### axis : Must be in the range [-rank(tensor), rank(tensor)).

In [82]:
reversed_t = tf.reverse(tensor, axis=[4])

print(sess.run(reversed_t))

ValueError: 'axis'[0] = 4 is out of valid range [0, 3 for 'ReverseV2_11' (op: 'ReverseV2') with input shapes: [1,2,3,4], [1] and with computed input tensors: input[1] = <4>.

## roll

In [85]:
tensor = [0,1,2,3]
res = tf.roll(tensor, shift=2, axis=0) 

print(sess.run(res))

[2 3 0 1]


In [91]:
tensor = tf.convert_to_tensor([[[[ 0,  1,  2,  3],
                 [ 4,  5,  6,  7],
                 [ 8,  9, 10, 11]],
                [[12, 13, 14, 15],
                 [16, 17, 18, 19],
                 [20, 21, 22, 23]]]])

res = tf.roll(tensor, shift=2, axis=3) 

print(sess.run(res))

[[[[ 2  3  0  1]
   [ 6  7  4  5]
   [10 11  8  9]]

  [[14 15 12 13]
   [18 19 16 17]
   [22 23 20 21]]]]


## meshgrid

In [148]:
x = [1, 2, 3]
y = [4, 5, 6]
X, Y = tf.meshgrid(x, y)

print(sess.run(X))
print(sess.run(Y))

[[1 2 3]
 [1 2 3]
 [1 2 3]]
[[4 4 4]
 [5 5 5]
 [6 6 6]]


## tile

In [35]:
x = [1, 2, 3]
multiples = [3]

res = tf.tile(
    x,
    multiples,
    name=None
)

sess.run(res)

array([1, 2, 3, 1, 2, 3, 1, 2, 3], dtype=int32)

## stack

In [37]:
x = tf.constant([1, 4])
y = tf.constant([2, 5])
z = tf.constant([3, 6])

res1 = tf.stack([x, y, z], axis=0)  # [[1, 4], [2, 5], [3, 6]] (Pack along first dim.)
res2 = tf.stack([x, y, z], axis=1)  # [[1, 2, 3], [4, 5, 6]]

print(sess.run(res1))
print(sess.run(res2))

[[1 4]
 [2 5]
 [3 6]]
[[1 2 3]
 [4 5 6]]


## unstack

## broadcast_to

In [38]:
x = tf.constant([1, 2, 3])
y = tf.broadcast_to(x, [3, 3])

sess.run(y)

array([[1, 2, 3],
       [1, 2, 3],
       [1, 2, 3]], dtype=int32)

In [39]:
x = tf.constant([1, 2, 3])
y = tf.broadcast_to(x, [4, 3])

sess.run(y)

array([[1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3]], dtype=int32)

In [40]:
x = tf.constant([1, 2, 3])
y = tf.broadcast_to(x, [4, 4])

sess.run(y)

ValueError: Dimensions must be equal, but are 3 and 4 for 'BroadcastTo_2' (op: 'BroadcastTo') with input shapes: [3], [2] and with input tensors computed as partial shapes: input[1] = [4,4].

# scatter

## scatter_add
https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/scatter_add

In [16]:
tgt = np.array([0., 1., 0.5, 0.6], dtype=float)
tsr = tf.Variable(tgt)

add_some = np.array([200., 100., -10.], dtype=float)

res = tf.scatter_add(
    tsr,
    indices=[3,1,0],
    updates=add_some,
    use_locking=False,
    name=None
)

sess.run(tsr.initializer)
sess.run(res)

array([-10. , 101. ,   0.5, 200.6])

In [19]:
tgt = np.array([0., 1., 0.5, 0.6], dtype=float)
tsr = tf.Variable(tgt)

sub_some = np.array([200., 100., -10.], dtype=float)

res = tf.scatter_sub(
    tsr,
    indices=[3,1,0],
    updates=sub_some,
    use_locking=False,
    name=None
)

sess.run(tsr.initializer)
sess.run(res)

array([  10. ,  -99. ,    0.5, -199.4])

## scatter_mul

In [18]:
tgt = np.array([0., 1., 0.5, 0.6], dtype=float)
tsr = tf.Variable(tgt)

mul_some = np.array([2., 10., -1.], dtype=float)

res = tf.scatter_mul(
    tsr,
    indices=[3,1,0],
    updates=mul_some,
    use_locking=False,
    name=None
)

sess.run(tsr.initializer)
sess.run(res)

array([-0. , 10. ,  0.5,  1.2])

## scatter_div

In [17]:
tgt = np.array([0., 1., 0.5, 0.6], dtype=float)
tsr = tf.Variable(tgt)

div_some = np.array([2., 10., -1.], dtype=float)

res = tf.scatter_div(
    tsr,
    indices=[3,1,0],
    updates=div_some,
    use_locking=False,
    name=None
)

sess.run(tsr.initializer)
sess.run(res)

array([-0. ,  0.1,  0.5,  0.3])

## scatter_max

In [20]:
tgt = np.array([0., 1., 0.5, 0.6], dtype=float)
tsr = tf.Variable(tgt)

max_some = np.array([2., 10., -1.], dtype=float)

res = tf.scatter_max(
    tsr,
    indices=[3,1,0],
    updates=max_some,
    use_locking=False,
    name=None
)

sess.run(tsr.initializer)
sess.run(res)

array([ 0. , 10. ,  0.5,  2. ])

## scatter_min

In [21]:
tgt = np.array([0., 1., 0.5, 0.6], dtype=float)
tsr = tf.Variable(tgt)

min_some = np.array([2., 10., -1.], dtype=float)

res = tf.scatter_min(
    tsr,
    indices=[3,1,0],
    updates=min_some,
    use_locking=False,
    name=None
)

sess.run(tsr.initializer)
sess.run(res)

array([-1. ,  1. ,  0.5,  0.6])

## scatter_nd
- [reference](https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/scatter_nd)
- scatter_nd 는 다른 scatter_XX 와 다르다.

```
tf.scatter_nd(
    indices,
    updates,
    shape,
    name=None
)
```

![아잇](https://www.tensorflow.org/images/ScatterNd1.png)

In [28]:
indices = [[4], [3], [1], [7]]
updates = [9, 10, 11, 12]
shape = [8]
scatter = tf.scatter_nd(indices, updates, shape)

sess.run(scatter)

array([ 0, 11,  0, 10,  9,  0,  0, 12], dtype=int32)

In [30]:
indices = tf.constant([[4], [3], [1], [7]])
updates = tf.constant([9, 10, 11, 12])
shape = tf.constant([8])
scatter = tf.scatter_nd(indices, updates, shape)

sess.run(scatter)

array([ 0, 11,  0, 10,  9,  0,  0, 12], dtype=int32)

![eich](https://www.tensorflow.org/images/ScatterNd2.png)

In [34]:
indices = tf.constant([[0], [2]])
updates = tf.constant([[[5, 5, 5, 5], [6, 6, 6, 6],
                        [7, 7, 7, 7], [8, 8, 8, 8]],
                       [[9, 9, 9, 9], [1, 1, 1, 1],
                        [9, 9, 9, 9], [1, 1, 1, 1]]])
shape = tf.constant([4, 4, 4])
scatter = tf.scatter_nd(indices, updates, shape)

sess.run(scatter)

array([[[5, 5, 5, 5],
        [6, 6, 6, 6],
        [7, 7, 7, 7],
        [8, 8, 8, 8]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[9, 9, 9, 9],
        [1, 1, 1, 1],
        [9, 9, 9, 9],
        [1, 1, 1, 1]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]]], dtype=int32)

# padding

## pad
https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/pad

In [57]:
t = tf.constant([[1, 2, 3], [4, 5, 6]])
paddings = tf.constant([[1, 1,], [2, 2]])
# 'constant_values' is 0.
# rank of 't' is 2.
res1 = tf.pad(t, paddings, "CONSTANT")  # [[0, 0, 0, 0, 0, 0, 0],
                                        #  [0, 0, 1, 2, 3, 0, 0],
                                        #  [0, 0, 4, 5, 6, 0, 0],
                                        #  [0, 0, 0, 0, 0, 0, 0]]

res2 = tf.pad(t, paddings, "REFLECT")  # [[6, 5, 4, 5, 6, 5, 4],
                                       #  [3, 2, 1, 2, 3, 2, 1],
                                       #  [6, 5, 4, 5, 6, 5, 4],
                                       #  [3, 2, 1, 2, 3, 2, 1]]

res3 = tf.pad(t, paddings, "SYMMETRIC")  # [[2, 1, 1, 2, 3, 3, 2],
                                         #  [2, 1, 1, 2, 3, 3, 2],
                                         #  [5, 4, 4, 5, 6, 6, 5],
                                         #  [5, 4, 4, 5, 6, 6, 5]]
print(sess.run(res1))
print(sess.run(res2))
print(sess.run(res3))
            

[[0 0 0 0 0 0 0]
 [0 0 1 2 3 0 0]
 [0 0 4 5 6 0 0]
 [0 0 0 0 0 0 0]]
[[6 5 4 5 6 5 4]
 [3 2 1 2 3 2 1]
 [6 5 4 5 6 5 4]
 [3 2 1 2 3 2 1]]
[[2 1 1 2 3 3 2]
 [2 1 1 2 3 3 2]
 [5 4 4 5 6 6 5]
 [5 4 4 5 6 6 5]]


# mask

## sequence_mask

In [45]:
mask = tf.sequence_mask([1, 3, 2], 5)

sess.run(mask)

array([[ True, False, False, False, False],
       [ True,  True,  True, False, False],
       [ True,  True, False, False, False]])

# py function

## py_func
https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/py_func#aliases

In [53]:
def my_func(x):
    # x will be a numpy array with the contents of the placeholder below
    return np.sinh(x)
input = tf.compat.v1.placeholder(tf.float32)
y = tf.compat.v1.py_func(my_func, [input], tf.float32)

sess.run(y, feed_dict={input:3})

10.017875